<h3>Tratar valores nulos nos dados de gatos utilizando média de outro dataset</h3>

In [1]:
import pandas as pd
dataset_dogs = pd.read_csv('../data/NYC_Dog_Licensing_Dataset.csv')

<b>Removendo colunas que não iremos utilizar</b>

In [2]:
dataset_dogs.drop(['RowNumber', 'CensusTract2010', 'NTA', 'CityCouncilDistrict', 'CommunityDistrict', 'CongressionalDistrict', 'StateSenatorialDistrict', 'LicenseIssuedDate', 'LicenseExpiredDate'], axis=1, inplace=True)
dataset_dogs.drop(dataset_dogs.index[67146:122203], axis=0, inplace=True)

In [3]:
from datetime import datetime

# substituindo os valores do campo AnimalBirthMonth para retirar a hora deixa somente a data
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'], format="%d/%m/%Y %H:%M:%S %p")
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'])
dataset_dogs['AnimalBirthMonth'] = dataset_dogs['AnimalBirthMonth'].map(lambda x: datetime.strftime(x,"%d/%m/%Y"), na_action='ignore')

In [4]:
dataset_dogs.to_csv('../data/dogs_clean.csv')

In [5]:
import csv

dicionario_ceps = {}

with open('../data/ceps.txt', 'r') as file:
    csvreader = csv.reader(file, delimiter=';')
    for linha in csvreader:
        dicionario_ceps[linha[0]] = linha

In [6]:
import numpy as np
np.random.seed(77)

chaves_dicionario = np.array(list(dicionario_ceps.keys()))

def get_cep():
    key = np.random.choice(chaves_dicionario)
    info_cep = dicionario_ceps[f'{key}']
    cep = info_cep[0]
    return cep

def get_cidade(cep):
    return dicionario_ceps[cep][1]

dataset_dogs["ZipCode"] = dataset_dogs["ZipCode"].map(lambda _: get_cep())
dataset_dogs["Borough"] = dataset_dogs["ZipCode"].map(lambda cep: get_cidade(cep))

dataset_dogs.to_csv("../data/dogs_clean.csv")

In [7]:
import os

dir_atual = os.getcwd()
dir_destino = os.path.abspath(os.path.join(dir_atual, os.pardir, '/home/joseph/Documentos/sptech/tcc/grupo11-4cco-tcc/src/inteligencia_artificial/train/'))

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]
fotos = set(arquivos_jpg)

In [8]:
dataset_breed_dogs = pd.read_csv('../../data/labels.csv')

In [9]:
dataset_breed_dogs_backup = dataset_breed_dogs

In [10]:
import base64

def is_base64(s):
    try:
        base64.b64decode(s)
        return True
    except Exception:
        return False

<h3>Adequação das raças de cachorros</h3>

In [11]:
breed_sample = dataset_breed_dogs.sample(n=len(dataset_dogs), replace=True)

# redefina o índice do DataFrame de raças
breed_sample = breed_sample.reset_index(drop=True)

# atualize a coluna 'BreedName' do DataFrame de cães com a coluna 'breed' do DataFrame de raças amostrado
dataset_dogs['BreedName'] = breed_sample['breed']

# salve o DataFrame atualizado em um novo arquivo CSV
dataset_dogs.to_csv('../../data/dogs_clean.csv', index=False)

In [12]:
dataset_dogs['BreedName'] = dataset_dogs['BreedName'].replace('Unknown', pd.NA)
breed_mean = dataset_dogs['BreedName'].dropna().mode()[0]
dataset_dogs['BreedName'] = dataset_dogs['BreedName'].fillna(breed_mean)

dataset_dogs.to_csv('../../data/dogs_clean.csv')

In [13]:
dataset_dogs['idMd5Image'] = None

In [14]:
dataset_dogs.head(3)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,idMd5Image
0,SHADOW,M,01/01/2000,silky_terrier,Gravataí/RS,94055030,None
1,ROCCO,M,10/01/2011,bull_mastiff,Várzea Grande/MT,78115646,None
2,LUIGI,M,09/01/2005,pembroke,Mossoró/RN,59633636,None


In [15]:
from tqdm import tqdm

async def find_image_from_name_dog(breed_name, dataset_breed_dogs, arquivos_jpg, dir_destino):
    if breed_name == 'Unknown' or not breed_name:
        return None, dataset_breed_dogs
    breed_name_lower = breed_name.lower()
    breeds_founded = dataset_breed_dogs[dataset_breed_dogs['breed'].str.lower().str.contains(breed_name_lower, case=False, regex=False)]
    
    if breeds_founded.empty:
        dataset_breed_dogs = dataset_breed_dogs_backup
        return None, dataset_breed_dogs
    
    first_breed = breeds_founded.iloc[0]
    df = dataset_breed_dogs.drop(breeds_founded.index[0])
    info_breed = first_breed.to_dict()
    # indice_arquivo = arquivos_jpg.index(f"{info_breed['id']}.jpg")
    
    return info_breed["id"], df


async def main(dataset_dogs, dataset_breed_dogs, arquivos_jpg, dir_destino):
    for index, row in tqdm(dataset_dogs.iterrows(), total=len(dataset_dogs)):
        md5, df = await find_image_from_name_dog(row['BreedName'], dataset_breed_dogs, arquivos_jpg, dir_destino)
        dataset_breed_dogs = df
        if not row["idMd5Image"]:
            dataset_dogs.loc[index, 'idMd5Image'] = md5


In [16]:
await main(dataset_dogs, dataset_breed_dogs, arquivos_jpg, dir_destino)

100%|██████████| 67146/67146 [05:50<00:00, 191.42it/s]


In [17]:
dataset_dogs.to_csv('../../data/dogs_clean.csv')

In [18]:
dataset_dogs.head(10)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,idMd5Image
0,SHADOW,M,01/01/2000,silky_terrier,Gravataí/RS,94055030,0964a2558fc9aa293a6a934d49f64968
1,ROCCO,M,10/01/2011,bull_mastiff,Várzea Grande/MT,78115646,02ad779f2721db9364d3bb68f5580582
2,LUIGI,M,09/01/2005,pembroke,Mossoró/RN,59633636,07e12e8064b6b17a94608ba5fd78a72e
3,PETUNIA,F,08/01/2013,cardigan,Mossoró/RN,59615377,031d3353588a81b42bcae74ff19571b0
4,ROMEO,M,10/01/2008,rhodesian_ridgeback,Itaguaí/RJ,23840240,019ff93e03802e661577b5869e099dcb
5,BRANDY,M,01/01/2004,tibetan_mastiff,São Bernardo do Campo/SP,09821190,0a8b6985bbe58a58909b5ef0a7d5a1aa
6,SAM,M,05/01/2011,chihuahua,Florianópolis/SC,88090650,0206c12e8984e3c8a166cc272de25d6f
7,MAY,F,05/01/2004,soft-coated_wheaten_terrier,Canoas/RS,92032694,0440e760e55c4d6bed536fbc0273801a
8,RUBY,F,04/01/2010,clumber,Vitória da Conquista/BA,45066012,0326344e0d5181130c28d25edd5627e4
9,LEO,M,01/01/2014,lhasa,Porto Alegre/RS,91420400,0311a6a51a414ba91f3ad8055170baa1


In [19]:
dataset_dogs.to_csv('../../data/dogs_clean.csv')